In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import requests
import time

In [ ]:


# 初始化 Selenium WebDriver
driver = webdriver.Chrome()  # 這裡使用 Chrome 瀏覽器，請確保已安裝 ChromeDriver

try:
    # 1. 打開網站 A
    driver.get("https://cleartest.twse.com.tw/etf/logon.jsp")

    # 2. 找到登入欄位並輸入帳號密碼
    username = driver.find_element(By.NAME, "ETF ID")
    user_id = driver.find_element(By.NAME,'用戶ID')
    password = driver.find_element(By.NAME, "密碼")
    
    username.send_keys("PGIM88")
    user_id.send_keys('PGIM8802')
    password.send_keys("Aa202410")

    
    # 提交登入表單
    password.send_keys(Keys.RETURN)

    # 等待頁面加載，並確認已成功登入
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "welcome-message")))

    # 3. 到 B 分頁
    driver.get("https://cleartest.twse.com.tw/etf/fileTransfer/SaveRecord.jsp")

    # 等待 B 分頁的表格加載完成
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "table")))

    # 獲取頁面源代碼
    page_source = driver.page_source

    # 使用 BeautifulSoup 解析頁面源代碼
    soup = BeautifulSoup(page_source, 'html.parser')

    # 4. 查找表格中的關鍵字，並找到對應的下載連結
    table = soup.find('table')
    rows = table.find_all('tr')
    
    download_url = None
    keyword = "your_keyword"
    for row in rows:
        cells = row.find_all('td')
        for cell in cells:
            if keyword in cell.text:
                # 找到包含關鍵字的行，獲取下載連結
                download_link = row.find('M21', href=True)
                if download_link:
                    download_url = download_link['href']
                    break
        if download_url:
            break

    # 如果找到下載連結，下載檔案
    if download_url:
        # 有些下載連結可能是相對路徑，這裡將其轉換為絕對路徑
        download_url = requests.compat.urljoin(driver.current_url, download_url)
        
        # 發送請求下載文件
        response = requests.get(download_url)
        if response.status_code == 200:
            with open('downloaded_file.ext', 'wb') as file:
                file.write(response.content)
            print("文件下載成功！")
        else:
            print("下載失敗，狀態碼：", response.status_code)
    else:
        print("未找到包含關鍵字的下載連結。")

finally:
    # 關閉瀏覽器
    driver.quit()
